In [18]:
import os
import torch
import torchvision
import numpy as np
import pandas as pd
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from IPython.display import display
import matplotlib.pyplot as plt
import torch.nn as nn
from tqdm import tqdm
import seaborn as sns
from PIL import Image
import random
import torchinfo
from torchinfo import summary
import torchvision.models as models
from torch import optim
from torch.optim import lr_scheduler

In [20]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

prep1 = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
        ])

In [21]:
model_vgg = models.vgg16(pretrained=True)

c:\Users\USER\.virtualenvs\bored_emotion_classifier-pq8v_Yyi\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\USER\.virtualenvs\bored_emotion_classifier-pq8v_Yyi\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [22]:
data_dir = 'data/cats_dogs_sample/'

dsets = {x: ImageFolder(os.path.join(data_dir, x), prep1)
         for x in ['train', 'test']}

dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=64,
                                               shuffle=False, num_workers=6)
                for x in ['train', 'test']}
dset_sizes = {x: len(dsets[x]) for x in ['train', 'test']}
dset_classes = dsets['train'].classes

In [23]:
model_vgg = models.vgg16(pretrained=True)
device = torch.device("cpu")
model_vgg = model_vgg.to(device)

c:\Users\USER\.virtualenvs\bored_emotion_classifier-pq8v_Yyi\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\USER\.virtualenvs\bored_emotion_classifier-pq8v_Yyi\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [24]:
for param in model_vgg.features.parameters():
    param.requires_grad = False
model_vgg.classifier[6].out_features = 2

In [25]:
criterion = nn.CrossEntropyLoss()
lr = 0.01
optimizer = torch.optim.SGD(model_vgg.classifier.parameters(),lr=lr)

In [26]:
from torch.autograd import Variable

In [28]:
# %%time
# train=True
# for epoch in range(2):
#         #batches = data_gen(conv_feat=conv_feat,labels=labels,shuffle=shuffle)
#         total = 0
#         running_loss = 0.0
#         running_corrects = 0
#         for inputs,classes in dset_loaders['train']:
#             inputs , classes = Variable(inputs),Variable(classes)
#             # inputs = inputs.view(inputs.size(0), -1)
#             outputs = model_vgg(inputs)
#             loss = criterion(outputs,classes)           
#             if train:
#                 if optimizer is None:
#                     raise ValueError('Pass optimizer for train mode')
#                 optimizer = optimizer
#                 optimizer.zero_grad()
#                 loss.backward()
#                 optimizer.step()
#             _,preds = torch.max(outputs.data,1)
#             # statistics
#             running_loss += loss.data[0]
#             running_corrects += torch.sum(preds == classes.data)
#         epoch_loss = running_loss / dset_sizes['train']
#         epoch_acc = running_corrects / dset_sizes['train']
#         print('Loss: {:.4f} Acc: {:.4f}'.format(
#                      epoch_loss, epoch_acc))

In [ ]:
def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer):
    model.train()
    train_loss, train_acc = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        correct_predictions = (y_pred_class == y).sum().item()
        total_predictions = len(y_pred) 
        accuracy = correct_predictions / total_predictions
        train_acc += accuracy
        # train_acc += (y_pred_class == y).sum().item()/len(y_pred)
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

In [ ]:
torch.save(model_vgg.state_dict(),"model/cat_dog_full_vgg_finetune_epoch.pth")